## 联盟选择

实验设定：
- 10个客户端
- 计算相似度的方法与论文描述相同

- 第一节介绍了FedAvg在1-20个epochs训练的结果，可以看到epochs越大，相似度计算越准确。
- 第二节补充了FedAvg在21-40个epochs训练的结果，用于探究结果是否会收敛，结果也是相同的。
- 实验结果与上次讨论的猜测吻合：**前期计算，性能并不稳定，得到的相似度也不准确；随着训练轮次的增加，相似度的结果就能符合我们的预期。不同数据集下，相同分布的相似度更高。**

### 1-20 epochs的结果

In [13]:
import numpy as np
from scipy.special import softmax
import sys
sys.path.append("/data/xuyc/workspace/fedvar_inc")
from global_tools import powerset, load_pkl



# e = 10
for e in range(1,21):
    print(f"{e}: ", end=" ")
    exp_names = ['cifar10_dFr1.0_nUs10_dir10.00_f1.0_e20_lEp1_s1', 'cifar10_dFr1.0_nUs10_niid-label_dAl10.0_f1.0_cPe2_e20_lEp1_s1']
    run_names = ['fedavg_cnn_l0.01_r0_tBa-1']
    results = {}
    for id,name in enumerate(exp_names):
        for run_name in run_names:
            root = f'/data/xuyc/save/FedVar/exp1.7.1_pretrain_cifar10/{name}/{run_name}/nr1/perf/'

            num_users = 10
            combinations_list = list(powerset(list(range(num_users))))

            # sv_users = np.zeros((100,num_users))
            datas = []
            data = list(load_pkl(root+ ("%d.pkl" % e)).values()) 
            
            data = [i['global_test_acc'] for i in data]
            # sv_users[e-1] = compute_shapley_value(num_users, data)
            datas.append(data)
            datas = np.array(datas)
            datas = datas.squeeze()
            

            feats = [[] for _ in range(num_users-2)]
            combinations_list = list(powerset(list(range(num_users))))
            for ind, comb in enumerate(combinations_list):
                if len(comb) >= 1 and len(comb) < num_users-1:
                    feats[len(comb)-1].append(datas[ind])

            vec = np.zeros(num_users-2)
            for ind, f in enumerate(feats):
                vec[ind] = np.std(f)
            results[name + run_name] = vec

            
    exp_names = ['cifar20_dFr1.0_nUs10_dir10.00_f1.0_e20_lEp1_s1', 'cifar20_dFr1.0_nUs10_niid-label_dAl10.0_f1.0_cPe2_e20_lEp1_s1']
    run_names = ['fedavg_cnn_l0.01_r0_tBa-1']

    for id, name in enumerate(exp_names):
        for run_name in run_names:
            # name = name.replace('e500','e1')
            root = f'/data/xuyc/save/FedVar/exp1.7.4_pretrain_cifar20/{name}/{run_name}/nr1/perf/'

            num_users = 10
            combinations_list = list(powerset(list(range(num_users))))

            # sv_users = np.zeros((100,num_users))
            datas = []
            data = list(load_pkl(root+ ("%d.pkl" % e)).values())
            
            data = [i['global_test_acc'] for i in data]
            # sv_users[e-1] = compute_shapley_value(num_users, data)
            datas.append(data)
            datas = np.array(datas)
            datas = datas.squeeze()
            

            feats = [[] for _ in range(num_users-2)]
            combinations_list = list(powerset(list(range(num_users))))
            for ind, comb in enumerate(combinations_list):
                if len(comb) >= 1 and len(comb) < num_users-1:
                    feats[len(comb)-1].append(datas[ind])

            vec = np.zeros(num_users-2)
            for ind, f in enumerate(feats):
                vec[ind] = np.std(f)
            results[name + run_name] = vec
    # plt.legend()
    # plt.show()

    n = np.array(list(results.values()))
    n_norm = (n / np.linalg.norm(n,ord=2,axis=1,keepdims=True))

    csv_results = []
    for i in range(n_norm.shape[0]):
        sims = (n_norm * n_norm[i]).sum(axis=1)
        csv_results.append(sims)
        if i <= 1:
            print(np.argmax(sims[2:]), end=" ")
        elif i > 1:
            print(np.argmax(sims[:2]), end=" ")
        # for j in sims:
        #     print("%.3f" % j, end=",")
        # print()
    print()

1:  1 0 1 0 
2:  1 1 1 0 
3:  1 1 0 1 
4:  1 0 1 1 
5:  1 1 1 1 
6:  0 1 1 1 
7:  0 1 1 1 
8:  0 1 0 1 
9:  0 1 0 1 
10:  0 1 0 1 
11:  0 1 0 1 
12:  0 1 0 1 
13:  0 1 1 1 
14:  0 1 0 1 
15:  0 1 0 1 
16:  0 1 1 1 
17:  0 1 0 1 
18:  0 1 0 1 
19:  0 1 0 1 
20:  0 1 1 1 


上述每行表示一个epoch
- 第1列：cifar10 + dir(10)
- 第2列：cifar10 + label(2)
- 第3列：cifar20 + dir(10)
- 第4列：cifar20 + label(2)

### epoch21-40

In [7]:
import numpy as np
from scipy.special import softmax
import sys
sys.path.append("/data/xuyc/projects/fedvar_inc")
from global_tools import powerset, load_pkl

epoch_list = range(21,41)

# e = 10
for e in epoch_list:
    print(f"{e}: ", end=" ")
    exp_names = ['cifar10_dFr1.0_nUs10_dir10.00_f1.0_e40_lEp1_s1', 'cifar10_dFr1.0_nUs10_niid-label_dAl10.0_f1.0_cPe2_e40_lEp1_s1']
    run_names = ['fedavg_cnn_l0.01_r0_tBa-1']
    results = {}
    for id,name in enumerate(exp_names):
        for run_name in run_names:
            root = f'/data/xuyc/save/FedVar/exp1.7.1_pretrain_cifar10/{name}/{run_name}/nr1/perf/'

            num_users = 10
            combinations_list = list(powerset(list(range(num_users))))

            # sv_users = np.zeros((100,num_users))
            datas = []
            data = list(load_pkl(root+ ("%d.pkl" % e)).values()) 
            
            data = [i['global_test_acc'] for i in data]
            # sv_users[e-1] = compute_shapley_value(num_users, data)
            datas.append(data)
            datas = np.array(datas)
            datas = datas.squeeze()
            

            feats = [[] for _ in range(num_users-2)]
            combinations_list = list(powerset(list(range(num_users))))
            for ind, comb in enumerate(combinations_list):
                if len(comb) >= 1 and len(comb) < num_users-1:
                    feats[len(comb)-1].append(datas[ind])

            vec = np.zeros(num_users-2)
            for ind, f in enumerate(feats):
                vec[ind] = np.std(f)
            results[name + run_name] = vec

            
    exp_names = ['cifar20_dFr1.0_nUs10_dir10.00_f1.0_e40_lEp1_s1', 'cifar20_dFr1.0_nUs10_niid-label_dAl10.0_f1.0_cPe2_e40_lEp1_s1']
    run_names = ['fedavg_cnn_l0.01_r0_tBa-1']

    for id, name in enumerate(exp_names):
        for run_name in run_names:
            name = name.replace('e500','e1')
            root = f'/data/xuyc/save/FedVar/exp1.7.4_pretrain_cifar20/{name}/{run_name}/nr1/perf/'

            num_users = 10
            combinations_list = list(powerset(list(range(num_users))))

            # sv_users = np.zeros((100,num_users))
            datas = []
            data = list(load_pkl(root+ ("%d.pkl" % e)).values())
            
            data = [i['global_test_acc'] for i in data]
            # sv_users[e-1] = compute_shapley_value(num_users, data)
            datas.append(data)
            datas = np.array(datas)
            datas = datas.squeeze()
            

            feats = [[] for _ in range(num_users-2)]
            combinations_list = list(powerset(list(range(num_users))))
            for ind, comb in enumerate(combinations_list):
                if len(comb) >= 1 and len(comb) < num_users-1:
                    feats[len(comb)-1].append(datas[ind])

            vec = np.zeros(num_users-2)
            for ind, f in enumerate(feats):
                vec[ind] = np.std(f)
            results[name + run_name] = vec
    # plt.legend()
    # plt.show()

    n = np.array(list(results.values()))
    n_norm = (n / np.linalg.norm(n,ord=2,axis=1,keepdims=True))

    csv_results = []
    for i in range(n_norm.shape[0]):
        sims = (n_norm * n_norm[i]).sum(axis=1)
        csv_results.append(sims)
        if i <= 1:
            print(np.argmax(sims[2:]), end=" ")
        elif i > 1:
            print(np.argmax(sims[:2]), end=" ")
        # for j in sims:
        #     print("%.3f" % j, end=",")
        # print()
    print()

21:  0 1 0 1 
22:  0 1 0 1 
23:  0 1 0 1 
24:  0 1 0 1 
25:  0 1 0 1 
26:  0 1 0 1 
27:  0 1 0 1 
28:  0 1 1 1 
29:  0 1 0 1 
30:  0 1 0 1 
31:  0 1 0 1 
32:  0 1 0 1 
33:  0 1 0 1 
34:  0 1 0 1 
35:  0 1 0 1 
36:  0 1 1 1 
37:  0 1 0 1 
38:  0 1 0 1 
39:  0 1 0 1 
40:  0 1 0 1 
